In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [30]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT SUM(CRIM IS NULL) as Crim,
       SUM(ZN IS NULL) as Zn,
       SUM(INDUS IS NULL) as Indus,
       SUM(CHAS IS NULL) as Chas,
       SUM(NOX IS NULL) as Nox 
    FROM boston
    """,
    con,
)

,Crim,Zn,Indus,Chas,Nox
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT count(distinct CRIM) as Crim,
       count(distinct ZN) as Zn,
       count(distinct INDUS) as Indus,
       count(distinct CHAS) as Chas,
       count(distinct NOX) as Nox
    FROM Boston
    """,
    con,
)

,Crim,Zn,Indus,Chas,Nox
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
from statistics import median


pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)
df = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']].median()
df

CRIM     0.25651
ZN       0.00000
INDUS    9.69000
CHAS     0.00000
NOX      0.53800
dtype: float64

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [62]:
pd.read_sql(
    """
with rm as (
SELECT AVG(RM) as avg
FROM boston
WHERE MEDV = (SELECT MAX(MEDV) FROM boston))
SELECT 25 as piece,
(SELECT avg FROM rm) - (SELECT AVG(RM) FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 25)) as total
UNION
SELECT 50 as piece,
(SELECT avg FROM rm) - (SELECT AVG(RM) FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 50)) as total
UNION
SELECT 100 as piece,
(SELECT avg FROM rm) - (SELECT AVG(RM) FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 100)) as total
UNION
SELECT 200 as piece,
(SELECT avg FROM rm) - (SELECT AVG(RM) FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 200)) as total
UNION
SELECT 300 as piece,
(SELECT avg FROM rm) - (SELECT AVG(RM) FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 300)) as total
""",
con,
)


,piece,total
0,25,1.736160
1,50,1.730760
2,100,1.596880
3,200,1.572295
4,300,1.511773


Видна зависимость комнат на стоимость

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [10]:
pd.read_sql(
    """
    SELECT LSTAT, rank 
    FROM (
    SELECT LSTAT, MEDV, RANK() OVER (order by LSTAT) as rank 
    FROM boston)
    WHERE MEDV = (SELECT MAX(MEDV) 
    FROM boston)
    limit 5
    """,
    con,
)

,LSTAT,rank
0,1.73,1
1,1.92,2
2,2.88,6
3,2.96,8
4,2.97,9


Чем выше статус, тем дороже недвижимость

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [11]:
pd.read_sql(
    """
    SELECT CHAS, round(AVG(MEDV)) 
    FROM boston
    GROUP BY CHAS
    """,
    con,
)

,CHAS,round(AVG(MEDV))
0,0.0,22.0
1,1.0,28.0


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [12]:
pd.read_sql(
    """
    SELECT CRIM, ZN, INDUS, NOX from(
    select ((SELECT AVG(CRIM)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(CRIM)   FROM boston WHERE CHAS = 0)) as CRIM
    , ((SELECT AVG(ZN)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(ZN)   FROM boston WHERE CHAS = 0)) as ZN
    , ((SELECT AVG(INDUS)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(INDUS)   FROM boston WHERE CHAS = 0)) as INDUS
    , ((SELECT AVG(NOX)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(NOX)   FROM boston WHERE CHAS = 0)) as NOX)
    """,
    con,
)

,CRIM,ZN,INDUS,NOX
0,-1.871545,-3.920534,1.69995,0.041609


Среднее выше у Доли акров неторгового бизнеса на город и Концентрация оксидов азота (частей на 10 миллионов), когда дом граничит с рекой!

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [55]:
pd.read_sql(
    """
    SELECT percentile * 10 as p, avg(INDUS) as Indus, avg(NOX) as Nox
    FROM (
    SELECT ntile(10) over (order by INDUS, NOX) as percentile, INDUS, NOX
    FROM boston)
    group by 1
    """,
    con,)


,p,Indus,Nox
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.564920
7,80,18.100000,0.684880
8,90,18.307200,0.714620
9,100,21.938800,0.692860


Концентрации оксидов азота зависит от доли промышленной застройки.